In [1]:
import random
import numpy as np
import cirq
import warnings
from cirq import NamedQubit, Circuit, identity_each, ZZ, H, X, depolarize, rx, DensityMatrixSimulator
from typing import List, Tuple
from typing import Callable
from scipy.optimize import minimize
# Suppress Cirq warnings
warnings.filterwarnings("ignore", category=UserWarning, module="cirq")
my_list = ['X', 'Y', 'Z']
state0=np.array([[1],[0]])
state1=np.array([[0],[1]])
list_state=[state0,state1]
#根据量子线路比特数和要采用的随机线路的数量生成要添加的具体随机幺正操作
def gen_list_unitary(nqubits,nsamp):
    list_unitary=[]
    for l in range(nsamp):
        string1=''
        for k in range(nqubits):
            selected_element = random.choice(my_list)
            string1 = string1 + selected_element
        list_unitary.append(string1)
    return list_unitary
II=np.array([[1,0],[0,1]])
X=np.array([[0,1],[1,0]])
Y=np.array([[0,-1j],[1j,0]])
Z=np.array([[1,0],[0,-1]])
Hm=np.sqrt(0.5)*np.array([[1,1],[1,-1]])
Sdag=np.array([[1, 0], [0, -1j]])
def get_matrix(gate):
    if gate=='X':
        return Hm
    if gate=='Y':
        return Hm@Sdag
    if gate=='Z':
        return II

#获得幺正操作的矩阵形式
def get_unitary(string):
    u1=get_matrix(string[0])
    for k in range(1,len(string)):
        u1=np.kron(u1,get_matrix(string[k]))
    return u1
#根据字符串生成量子态
def gen_state(string):
    state_in1=list_state[int(string[0])]
    for k in range(1,len(string)):
        state_in1=np.kron(state_in1,list_state[int(string[k])])
    return state_in1
#计算量子态之间的迹距离
def trace_distance(rho, sigma):
    diff = rho - sigma
    sqrt_diff = np.sqrt(np.dot(diff.conj().T, diff))
    trace_dist = np.trace(sqrt_diff) / 2.0
    return trace_dist

In [3]:
def classical_shadow(nqubits,circuit,list_unitary,nsamp,shots):
    shadow = []
    #根据生成的list_unitary在给定的量子线路后施加随机pauli操作
    for l in range(len(list_unitary)):
        circ = circuit.copy()
        qubits = list(circuit.all_qubits())
        for k in range(nqubits):
            if list_unitary[l][k] == 'X':
                circ.append(cirq.H(qubits[k]))
            elif list_unitary[l][k] == 'Y':
                circ.append(cirq.H(qubits[k]))
                circ.append(cirq.S(qubits[k])**-1)
            elif list_unitary[l][k] == 'Z':
                circ.append(cirq.I.on(qubits[k]))

        circ.append(cirq.measure(*qubits, key='result'))
        simulator = cirq.Simulator()
        result = simulator.run(circ, repetitions=shots)
        states = result.measurements['result']
        #根据结果重建量子态
        for state in states:
            rho0 = 3 * np.conjugate(get_matrix(list_unitary[l][0]).T) @ list_state[int(state[0])] @ \
                   np.conjugate(list_state[int(state[0])].T) @ get_matrix(list_unitary[l][0]) - II

            for k in range(1, nqubits):
                rho1 = 3 * np.conjugate(get_matrix(list_unitary[l][k]).T) @ list_state[int(state[k])] @ \
                       np.conjugate(list_state[int(state[k])].T) @ get_matrix(list_unitary[l][k]) - II
                rho0 = np.kron(rho0, rho1)
            shadow.append(rho0)
        shadow_state=shadow[0]
    for k in range(1,len(shadow)):
        shadow_state=shadow_state+shadow[k]
    shadow_state=shadow_state/len(shadow)
    return shadow_state

In [4]:
square_graph = [(0, 1), (1, 2), (2, 3), (3,0)]
def qaoa_hamiltonian(graph: List[Tuple[float]]) -> np.ndarray:
    """Returns the cost Hamiltonian associated to the input graph.
    """
    # Get all the nodes of the graph
    nodes = list({node for edge in graph for node in edge})
    # Initialize the qubits. One for each node.
    qreg = [NamedQubit(str(nn)) for nn in nodes]
    # Define the Hamiltonian as a NumPy array
    np_identity = np.eye(2 ** len(nodes))
    zz_terms = np.real([Circuit([identity_each(*qreg), ZZ(qreg[i], qreg[j])]).unitary() for i, j in graph])
    local_terms = [-0.5 * (np_identity - zz_term) for zz_term in zz_terms]
    return sum(local_terms)

In [5]:
def qaoa_ansatz(graph: List[Tuple[float]], params: List[float]) -> Circuit:
    """Generates a QAOA circuit associated to the input graph, for
    a specific choice of variational parameters.

    Args:
        graph: The input graph.
        params: The variational parameters.

    Returns:
        The QAOA circuit.
    """

    # Get the list of unique nodes from the list of edges
    nodes = list({node for edge in graph for node in edge})

    # Initialize the qubits
    qreg = [NamedQubit(str(nn)) for nn in nodes]

    # Define the Hamiltonian evolution (up to an additive and a multiplicative constant)
    def h_step(beta: float) -> Circuit:
        return Circuit(cirq.ZZ(qreg[u], qreg[v]) ** (beta) for u, v in graph)

    # Define the mixing evolution (up to an additive and a multiplicative constant)
    def mix_step(gamma: float) -> Circuit:
        return Circuit(cirq.X(qq) ** gamma for qq in qreg)

    # State preparation layer
    circ = Circuit(H.on_each(qreg))

    # Apply QAOA steps
    num_steps = len(params) // 2
    betas, alphas = params[:num_steps], params[num_steps:]
    for beta, alpha in zip(betas, alphas):
        circ.append([h_step(beta) + mix_step(alpha)])
    return circ

In [6]:
betas = [0.1, 0.2]
alphas = [0.3, 0.4]
params = betas + alphas
qaoa_ansatz(square_graph, params)

0: ───H───ZZ─────────────────────────ZZ───────X^0.3───ZZ─────────────────────────ZZ───────X^0.4───
          │                          │                │                          │
1: ───H───ZZ^0.1───ZZ────────────────┼────────X^0.3───ZZ^0.2───ZZ────────────────┼────────X^0.4───
                   │                 │                         │                 │
2: ───H────────────ZZ^0.1───ZZ───────┼────────X^0.3────────────ZZ^0.2───ZZ───────┼────────X^0.4───
                            │        │                                  │        │
3: ───H─────────────────────ZZ^0.1───ZZ^0.1───X^0.3─────────────────────ZZ^0.2───ZZ^0.2───X^0.4───

In [7]:
def executor(obs: np.array, params: List[float],list_unitary,noise:float) -> float:
    """模拟带有级别为'noise'的去极化噪声的电路，并返回可观察值'obs'的期望值。
    """
    circuit_qaoa=qaoa_ansatz(square_graph, params)
    noisy_circuit=circuit_qaoa.with_noise(depolarize(p=noise))
    shadow_state=classical_shadow(nqubits=4,circuit=circuit_qaoa,list_unitary=list_unitary,nsamp=50,shots=1024)
    return np.real(np.trace(shadow_state @ obs))

In [8]:
betas = [0.1, 0.2]
alphas = [0.3, 0.4]
params = betas + alphas
nqubits=4
nsamp=50
hamiltonian = qaoa_hamiltonian(square_graph)
LIST_UNITARY=gen_list_unitary(nqubits,nsamp)
executor(obs=hamiltonian, params=params,list_unitary=LIST_UNITARY,noise=0.03)

-1.6399701192809306